# Setup libs and enviroment

In [ ]:
! pip install datasets transformers seqeval
! apt install git-lfs
! pip install -U accelerate
! pip install -U transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=a1e8d781e3d5ce619b0c73b5e806903debb2c8cc47d73d0ec4f147cb8b9ce39b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import gc
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
os.environ['PYTORCH_CUDA_ALLOC_CONF']="expandable_segments:True"

from google.colab import drive
drive.mount('/content/drive')

import transformers
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer, AutoTokenizer
print(transformers.__version__)

from huggingface_hub import notebook_login
notebook_login()

from transformers.utils import send_example_telemetry
send_example_telemetry("token_classification_notebook", framework="pytorch")

import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import pandas as pd
import numpy as np
import random
import datasets
from datasets import load_metric, DatasetDict, load_dataset

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

Mounted at /content/drive
4.39.0


cuda:0


# Input preparation

In [ ]:
text = "Поэтому при стабилизации используйте бордюрную  (ацетатную ленту), она предотвратит подсыхание бисквита."
input = tokenizer(text, is_split_into_words=False)

print("Пример обработки текста моделью\nТЕКСТ: ", text)
print("ТОКЕНЫ", tokenizer.convert_ids_to_tokens(input['input_ids']))
print("КОДИРОВАНЫЕ ТОКЕНЫ: ", input['input_ids'])


Пример обработки текста моделью
ТЕКСТ:  Поэтому при стабилизации используйте бордюрную  (ацетатную ленту), она предотвратит подсыхание бисквита.
ТОКЕНЫ ['[CLS]', 'поэтому', 'при', 'стабилизации', 'использу', '##ите', 'бордюр', '##ную', '(', 'ацет', '##ат', '##ную', 'ленту', ')', ',', 'она', 'предотвра', '##тит', 'подс', '##ых', '##ани', '##е', 'биск', '##вита', '.', '[SEP]']
КОДИРОВАНЫЕ ТОКЕНЫ:  [101, 2686, 711, 26668, 3097, 42699, 98051, 918, 160, 116019, 3788, 918, 34090, 158, 121, 960, 9691, 3883, 4800, 14927, 12309, 376, 96890, 34183, 126, 102]


# Models finetuning

In [ ]:
FINAL_MODELS = ['disk0dancer/ruBert-base-finetuned-pos', 'disk0dancer/ruBert-large-finetuned-pos', 'disk0dancer/ruRoberta-large-finetuned-pos']
MODELS = ("ai-forever/ruBert-base", "ai-forever/ruBert-large", "ai-forever/ruRoberta-large")
DATASETS = ("original", "resampled")
BATCH_SIZE = 16
MAX_LEN = 100
LR = 2e-5
metric = load_metric("seqeval", trust_remote_code=True)

<ipython-input-2-ed9387f090c4>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval", trust_remote_code=True)


In [ ]:

def capitalize(s:str)->str:
    return s[0].upper()+s[1:].lower()

with open('/content/drive/MyDrive/coursework3/tags.txt', 'r') as f:
    label_list=f.read().replace('”', '').replace('“', '').split('\n')

# all_labels = []
# for pref in ['B-', 'I-', 'S-', 'E-']:
#     for tag in label_list:
#         all_labels.append(pref+capitalize(tag))

all_labels = list(map(capitalize, label_list))
tag_to_ids = {k: v for k, v in zip(all_labels, range(1, len(all_labels)+1))}
ids_to_tag = {v: k for k, v in zip(all_labels, range(1, len(all_labels)+1))}

tag_to_ids['O']=-0
ids_to_tag[0]='O'

def tokenize(input_and_labels:str, tag_to_ids=tag_to_ids, max_len=MAX_LEN):
    input = tokenizer(input_and_labels["ru"],  truncation=True, padding='max_length', is_split_into_words=False,  max_length=max_len)
    input['input_ids'][0]=0
    input['input_ids'][-1]=0

    tags = input_and_labels["schema"].replace('”', '').replace('“', '').replace('desc', 'desс').split()
    tags_num = [len(tokenizer.convert_ids_to_tokens(tokenizer(w)["input_ids"]))-2 for w in input_and_labels["ru"].split()]

    labels = ['O']

    for t, n in zip(tags, tags_num):
        tag = capitalize(t)
        if n==1:
            # labels.append('S-'+tag)
            labels.append(tag)
        else:
            # labels.append('B-'+tag)
            labels.append(tag)

            # for _ in range(1, n-1):
                #  labels.append('I-'+tag)

            for _ in range(1, n):
                labels.append('O')

            # labels.append('E-'+tag)

    labels.append('O')
    while len(labels)!= len(input['input_ids']):
        labels.append('O')

    ids_labels = [tag_to_ids[tag] for tag in labels]

    return {"input_ids":input['input_ids'], "labels": ids_labels, "attention_mask":input["attention_mask"]}



def get_datasets(dataset_type):
    datsets_list = []
    for part in ('train', 'test', 'val'):
        df = pd.read_csv(f'/content/data-{dataset_type}-{part}.csv', sep='\t')
        df.columns = ['ru', 'schema']
        df['schema'] = df['schema'].apply(lambda x: str(x))
        df['ru'] = df['ru'].apply(lambda x: str(x))

        if part == "train":
            df = df.iloc[::5]

        ds = datasets.Dataset.from_pandas(df)
        ds_tok = ds.map(tokenize, batched=False).remove_columns(['ru', 'schema']).with_format("torch")
        datsets_list.append(ds_tok)
    return datsets_list



label_list = [0] + all_labels

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != 0]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != 0]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


def model_inference_metrics(trainer, dataset):
    predictions, labels, _ = trainer.predict(dataset)
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != 0]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != 0]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results


In [ ]:
# tokenize func + tag_to_ids + ids_to_tag
# compute metric func

for MODEL_NAME in MODELS:

    # get pretrained
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, max_length=MAX_LEN, truncation=True, padding=True)
    data_collator = DataCollatorForTokenClassification(tokenizer, max_length=MAX_LEN, padding='max_length', label_pad_token_id=0)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(list(ids_to_tag.keys())), id2label=ids_to_tag, label2id=tag_to_ids)

    # break
    for DATASET in DATASETS:
        # clean up rosources
        torch.cuda.empty_cache()
        gc.collect()
        print(MODEL_NAME, DATASET, end='\n\n\n')
        EPOCH = 5 if DATASET=="resampled" else 45

        train, val, test = get_datasets(DATASET)
        args = TrainingArguments(
            f"{MODEL_NAME.split('/')[-1]}-finetuned-pos",
            # MODEL_NAME,
            learning_rate=LR,
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,
            num_train_epochs=EPOCH,
            weight_decay=0.01,
            overwrite_output_dir=True,
            save_total_limit = 2,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
        )

        trainer = Trainer(
            model,
            args,
            train_dataset=train,
            eval_dataset=val,
            data_collator=data_collator,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        trainer.place_model_on_device=True

        trainer.train()

        trainer.evaluate()

        print(MODEL_NAME, DATASET, model_inference_metrics(trainer, test), sep='\n', end='\n'+'-'*50+'\n')
        with open('log.txt', 'a') as log:
            log.write('\n'.join([MODEL_NAME, DATASET, str(model_inference_metrics(trainer, test)), '\n'+'-'*50+'\n']))

        trainer.push_to_hub()


disk0dancer/ruBert-base-finetuned-pos resampled




Map:   0%|          | 0/3676 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


disk0dancer/ruRoberta-large-finetuned-pos
resampled
{'_': {'precision': 0.14285714285714285, 'recall': 0.028037383177570093, 'f1': 0.04687499999999999, 'number': 107}, 'ark/excl': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'ark/quest': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'art/intens': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 3}, 'art/neg': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'art/quest': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'articip/adj': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}, 'articip/v': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'ash': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 3}, 'bbr': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}, 'dj': {'precision': 0.5, 'recall': 0.05263157894736842, 'f1': 0.09523809523809525, 'number': 38}, 'dv': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'dv/action_desс': {'precision': 0

events.out.tfevents.1711117482.699a7512e26d.2634.10:   0%|          | 0.00/486 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

# Model config

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained('disk0dancer/ruBert-base-finetuned-pos')
model

config.json:   0%|          | 0.00/3.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e